# Clasificador de tweets con Insultos

A continuacion se muestra un analisis de tweets.
Basados en un listado de "Badwords", de google se analizan 5000 tweets y se clasifican asi:
Si el tweet contiene un insu

    Si el Tweet tiene algun insulto se almacena en el archivo: Classified_analisis_insultos.csv
    Si el Tweet no tiene insultos se almacena en el archivo: Classified_analisis_SINinsultos.csv

Al final de la practica se muestran los tweets con insultos y el insulto incluido en el tweet.


In [1]:
import pandas as pd
import numpy as np
import tweepy
from tweepy import OAuthHandler
from tweepy import API
from tweepy import Cursor
import datetime
import time
import csv
import json
import nltk
import re
from pprint import pprint

from nltk.tokenize import word_tokenize
from nltk.classify import NaiveBayesClassifier
import re

Obteniendo los datos de Twitter (5000 registros)

In [2]:
consumer_key = ''
consumer_secret = ''
access_token = ''
access_token_secret = ''

In [3]:
def search(hashtag, Max_Tweets, Recdata):

    auth = OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)
    
    api = API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
    data = Cursor(api.search, q=hashtag, lang='en').items(Max_Tweets)
    
    current_working_dir = "./data/"
    log_tweets = current_working_dir  + 'JSON_analisis_insultos.json'
    with open(log_tweets, 'w') as outfile:
        for tweet in data:
            Recdata.append(json.loads(json.dumps(tweet._json)))
            outfile.write(json.dumps(tweet._json))
            outfile.write("\n")
            
    # Podemos Crear dataset con esta informacion
    tweets = pd.DataFrame()
    # Campos del Tweet
#    tweets['created_at'] = map(lambda tweet: time.strftime('%Y-%m-%d %H:%M:%S', time.strptime(tweet['created_at'],'%a %b %d %H:%M:%S +0000 %Y')), Recdata)
#    tweets['user'] = map(lambda tweet: tweet['user']['screen_name'], Recdata)
    tweets['text'] = map(lambda tweet: tweet['text'].encode('utf-8'), Recdata)
   
    # Generar un archivo cvs
    tweets.to_csv('./data/CVS_analisis_insultos.csv', sep=';', header=False)

In [4]:
#Max_Tweets = 5000
#tweets_data = []
#hashtag = "news"
#
#search(hashtag, Max_Tweets, tweets_data)

Cargamos los datos que guardamos en el archivo de Tweet, para no ir siempre a Twitter, aunque el codigo utilizado para la descarga de tweets, esta al inicio de esta practica.

Primero cargaremos la lista de ** Insultos **

In [5]:
badwords_list = []
text_file = open('./data/bad-words.txt', 'r')
badwords_list = text_file.read().split('\n')

Ahora cargamos los Tweets

In [6]:
def limpiaNL_string(Texto):
    # definir la caracteres que debemos eliminar
    puntuacion = '\n'
    
    # Luego encontramos la posicion y la eliminamos
    cont = 0
    no_punct = ""
    for char in strT:
        if char not in puntuacion:
            no_punct = no_punct + char
    return no_punct

In [7]:
tweet_list = []
with open('./data/JSON_analisis_insultos.json', 'r') as f:
    for line in f:
        tweet = json.loads(line)
        tweet_list.append(tweet['text'].encode("utf-8"))

Tokenizamos la lista de tweets y creamos un nuevo lista

In [8]:
emoticons_str = r"""
    (?:
        [:=;] # Eyes
        [oO\-]? # Nose (optional)
        [D\)\]\(\]/\\OpP] # Mouth
    )"""
regex_str = [
    emoticons_str,
    r'<[^>]+>', # HTML tags
    r'(?:@[\w_]+)', # @-mentions
    r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", # hash-tags
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
    r'(?:(?:\d+,?)+(?:\.?\d+)?)', # numbers
    r"(?:[a-z][a-z'\-_]+[a-z])", # words with - and '
    r'(?:[\w_]+)', # other words
    r'(?:\S)' # anything else
]

tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)
emoticon_re = re.compile(r'^'+emoticons_str+'$', re.VERBOSE | re.IGNORECASE)

def tokenize(s):
    return tokens_re.findall(s)

def preprocess(s, lowercase=False):
    tokens = tokenize(s)
    if lowercase:
        tokens = [token if emoticon_re.search(token) else token.lower() for token in tokens]
    #return tokens
    return map(lambda x: x.lower(), tokens)

In [9]:
tweet_token = []
for line in tweet_list:
    tweet_token.append(preprocess(line))
print(tweet_token[200])

['rt', '@squeezereport', ':', 'inside', 'a', 'moneymaking', 'machine', 'like', 'no', 'other', 'https://t.co/xxpi562ej2', 'via', '@markets', 'get', 'on', 'a', 'level', 'playing', 'field', '!', '#stock', '#bl', '\xe2', '\x80', '\xa6']


Limpiaremos los tweets de las comillas ("")

In [10]:
# define punctuation
Classified_tweetsNP = []
punctuations = '''';\n"'''

for i in range(len(tweet_list)):
    my_str = tweet_list[i]
    # remove punctuation from the string
    no_punct = ""
    for char in my_str:
        if char not in punctuations:
            no_punct = no_punct + char
    Classified_tweetsNP.append(no_punct)

Clasificamos los Tweets ** Con insultos **

In [11]:
Classified_tweets = []
Classified_tweets_NI = []
cont_insult = 1
insulto_activo = 0
for i in range(len(tweet_list)):
    tweetTK_review = tweet_token[i]
    insult = 0
    insult_word = ""
    for tw_token in tweetTK_review:
        if tw_token.strip() in badwords_list:
            insult_word = tw_token
            insulto_activo = 1
            insult +=1
    # Creamos una nueva lista clasificada
    if insulto_activo > 0 and insult > 0:
        cont_insult += 1
        fwtweet = Classified_tweetsNP[i] + ';' + 'Insulto: ' + insult_word + '\n'
        Classified_tweets.append(fwtweet)
    else:
        Classified_tweets_NI.append(Classified_tweetsNP[i])
print('Total de Tweets con Insultos ----> ' + str(cont_insult) + ' de un total de: ' + str(len(tweet_list)) + \
      ' lo que es un ' + str((cont_insult*100)/len(tweet_list)) + '% de los tweets analizados')

Total de Tweets con Insultos ----> 205 de un total de: 5000 lo que es un 4% de los tweets analizados


Creamos archivos con las nuevas listas con los tweets clasificados

In [12]:
np.savetxt("./data/Classified_analisis_insultos.csv", Classified_tweets,fmt='%s')
np.savetxt("./data/Classified_analisis_SINinsultos.csv", Classified_tweets_NI,fmt='%s')

Muestra en Pantalla de los tweets con insultas (incluyendo RT)

In [13]:
for tweet in Classified_tweets:
    print tweet 

RT @MailOnline: Prostitute beats a man with a beer bottle after he uses a boiled egg to pay for sex in Zimbabwe https://t.co/IbdY6J37Tl htt…;Insulto: sex

RT @ShaunKing: Steve Bannons Alt-Right Movement goes FULL-NAZI this weekend. Literally, his supporters were doing Nazi salutes.https:/…;Insulto: nazi

KING: Alt-right goes full Nazi after Bannon lands White House job https://t.co/evbLRLm9R3;Insulto: nazi

RT @VieAvecLeigh: breaking fucking news : https://t.co/wg0OT0RroE;Insulto: fucking

RT @ShaunKing: This is real. VERY REAL. Alt-Right Neo Nazis loved by Trumps Chief Strategist say Hail Trump w/ Nazi Salute. https://…;Insulto: nazi

RT @EricBarrow: KING: Alt-right goes full Nazi after Bannon lands White House job https://t.co/yFXRv7iqAz;Insulto: nazi

RT @quadz08: America in the age of Trump. Where Actual Nazis question the personhood of Jews on national news. What the actual fuck.;Insulto: fuck

RT @ShaunKing: This is real. VERY REAL. Alt-Right Neo Nazis loved by Trumps Chief Strat